<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/VQA_BioGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BioGPT

SRC: https://github.com/microsoft/BioGPT.git

In [2]:
!pip -q install fairseq moses fastBPE sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 68.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!mkdir checkpoints
%cd checkpoints
!wget https://msramllasc.blob.core.windows.net/modelrelease/BioGPT/checkpoints/Pre-trained-BioGPT.tgz
!tar -zxvf Pre-trained-BioGPT.tgz
%cd ..

/content/checkpoints
--2023-02-04 13:38:35--  https://msramllasc.blob.core.windows.net/modelrelease/BioGPT/checkpoints/Pre-trained-BioGPT.tgz
Resolving msramllasc.blob.core.windows.net (msramllasc.blob.core.windows.net)... 20.209.34.164
Connecting to msramllasc.blob.core.windows.net (msramllasc.blob.core.windows.net)|20.209.34.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3414384105 (3.2G) [application/octet-stream]
Saving to: ‘Pre-trained-BioGPT.tgz’

Pre-trained-BioGPT. 100%[===================>]   3.18G  23.5MB/s    in 1m 54s  

2023-02-04 13:40:30 (28.5 MB/s) - ‘Pre-trained-BioGPT.tgz’ saved [3414384105/3414384105]

Pre-trained-BioGPT/
Pre-trained-BioGPT/checkpoint.pt
/content


In [9]:
# https://drive.google.com/file/d/1-kEJF9LS5_Fb-D-09lWMfzpx8169an8k/view?usp=sharing
!curl -L -s -o data.zip 'https://drive.google.com/uc?id=1-kEJF9LS5_Fb-D-09lWMfzpx8169an8k'
!unzip -q data.zip

In [1]:
import torch
from fairseq.models.transformer_lm import TransformerLanguageModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
m = TransformerLanguageModel.from_pretrained(
        "checkpoints/Pre-trained-BioGPT", 
        "checkpoint.pt", 
        "data",
        tokenizer='moses', 
        bpe='fastbpe', 
        bpe_codes="data/bpecodes",
        min_len=100,
        max_len_b=1024)
m.to(device)
src_tokens = m.encode("COVID-19 is a disease")
generate = m.generate([src_tokens], beam=5)[0]
output = m.decode(generate[0]["tokens"])
print(output)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


COVID-19 is a disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which was first identified in Wuhan, China, in December 2019 and has since spread to more than 200 countries and territories, including the United States, Canada, the United Kingdom, the United States of America, the United States of America, Australia, New Zealand, Canada, and the United States of America, as well as the rest of the world, including Europe, Asia, Africa, and South America.


Sample prediction-1:

In [ ]:
src_tokens = m.encode("HIV is a")
generate = m.generate([src_tokens], beam=2)[0]
output = m.decode(generate[0]["tokens"])
print(generate[0]['positional_scores'].shape)
print(output)

Sample prediction-2:

In [ ]:
src_tokens = m.encode("HIV is a virus from")
generate = m.generate([src_tokens], beam=2)[0]
output = m.decode(generate[0]["tokens"])
print(generate[0]['positional_scores'].shape)
print(output)

BioGPT for PubMedQA:
Dataset Src: https://pubmedqa.github.io/

Download pretrained weights:

In [21]:
!curl -L -s -o QA-PubMedQA-BioGPT.tgz 'https://msramllasc.blob.core.windows.net/modelrelease/BioGPT/checkpoints/QA-PubMedQA-BioGPT.tgz'

^C
